## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Bermeo,43.4209,-2.7215,48.00,100,20,2.30,ES,2021-01-18 01:15:29,few clouds
1,1,Avarua,-21.2078,-159.7750,86.00,62,100,8.05,CK,2021-01-18 01:15:29,overcast clouds
2,2,Muhororo,0.9381,30.7594,63.46,56,82,1.61,UG,2021-01-18 01:15:29,broken clouds
3,3,Qaanaaq,77.4840,-69.3632,-3.05,74,0,6.13,GL,2021-01-18 01:15:29,clear sky
4,4,La Pintada,5.7487,-75.6063,60.80,100,75,2.30,CO,2021-01-18 01:15:29,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
1,1,Avarua,-21.2078,-159.7750,86.00,62,100,8.05,CK,2021-01-18 01:15:29,overcast clouds
5,5,Rikitea,-23.1203,-134.9692,78.93,74,0,12.19,PF,2021-01-18 01:15:29,clear sky
11,11,The Valley,18.2170,-63.0578,78.80,78,75,9.22,AI,2021-01-18 01:15:30,broken clouds
16,16,Kahului,20.8947,-156.4700,75.20,83,75,4.61,US,2021-01-18 01:15:30,moderate rain
19,19,Kapaa,22.0752,-159.3190,80.60,65,75,17.27,US,2021-01-18 01:15:31,light rain
22,22,Acarau,-2.8856,-40.1200,80.20,78,100,12.95,BR,2021-01-18 01:15:31,overcast clouds
25,25,Saint-Philippe,-21.3585,55.7679,77.00,69,0,4.61,RE,2021-01-18 01:12:15,light rain
26,26,Ladario,-19.0047,-57.6017,75.87,94,100,2.26,BR,2021-01-18 01:12:19,heavy intensity rain
27,27,Porangatu,-13.4408,-49.1486,72.09,89,100,2.77,BR,2021-01-18 01:15:32,overcast clouds
28,28,Sola,-13.8833,167.5500,89.60,74,75,6.91,VU,2021-01-18 01:13:01,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        225
City           225
Lat            225
Lng            225
Max Temp       225
Humidity       225
Cloudiness     225
Wind Speed     225
Country        222
Date           225
Description    225
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Avarua,CK,86.00,overcast clouds,-21.2078,-159.7750,
5,Rikitea,PF,78.93,clear sky,-23.1203,-134.9692,
11,The Valley,AI,78.80,broken clouds,18.2170,-63.0578,
16,Kahului,US,75.20,moderate rain,20.8947,-156.4700,
19,Kapaa,US,80.60,light rain,22.0752,-159.3190,
22,Acarau,BR,80.20,overcast clouds,-2.8856,-40.1200,
25,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
26,Ladario,BR,75.87,heavy intensity rain,-19.0047,-57.6017,
27,Porangatu,BR,72.09,overcast clouds,-13.4408,-49.1486,
28,Sola,VU,89.60,broken clouds,-13.8833,167.5500,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Avarua,CK,86.00,overcast clouds,-21.2078,-159.7750,Paradise Inn
5,Rikitea,PF,78.93,clear sky,-23.1203,-134.9692,Pension Maro'i
11,The Valley,AI,78.80,broken clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
16,Kahului,US,75.20,moderate rain,20.8947,-156.4700,Maui Seaside Hotel
19,Kapaa,US,80.60,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
660,Lagunas,PE,76.37,overcast clouds,-5.2269,-75.6753,Hospedaje Eco
666,Caucaia,BR,82.40,moderate rain,-3.7361,-38.6531,M.O.S. Logística - Filial Fortaleza
668,Itarema,BR,80.19,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
669,Carutapera,BR,78.26,broken clouds,-1.1950,-46.0200,Lidera


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))